<a href="https://colab.research.google.com/github/LollaGavrishko/Competitiveness_calculation/blob/main/Q-Learning%20Taxi-V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reinforcement Q-Learning from Scratch in Python with OpenAI Gym
https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/

In [1]:

import gym
import time
from IPython.display import clear_output

env = gym.make("Taxi-v3").env


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [2]:
env.s = 328
env.reset()
env.render()
print(env.step(2))
time.sleep(10)
clear_output(wait=True)
env.render()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


(408, -1, False, {'prob': 1.0, 'action_mask': array([0, 1, 0, 0, 1, 0], dtype=int8)})


# Решение без алгоритма RL
Выполнение случайных действий из каждого состояния

In [3]:
epochs = 0
penalties, reward = 0, 0

frames = [] # для анимации

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1

    # Поместите каждый визуализированный кадр в dict для анимации.
    frames.append({
        'frame': env.render(mode='ansi'),
        'episode': '0',
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1


print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 124
Penalties incurred: 34


/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


## Printing frames

In [4]:
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Episode: {frame['episode']}")
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        time.sleep(.1)

In [5]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Episode: 0
Timestep: 124
State: 0
Action: 5
Reward: 20


#Поместите каждый визуализированный кадр в dict для анимации

In [6]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [7]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# гиперпараметры
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# Поместите каждый визуализированный кадр в словарь для анимации
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False

    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() #Исследуйте пространство действий
        else:
            action = np.argmax(q_table[state]) # Исследуйте пространство действий

        next_state, reward, done, info = env.step(action)

        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])

        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1

    if i % 100 == 0:
        clear_output(wait=True)
        print(f"эпизоды: {i}")

print("Завершение тренировки \n")

эпизоды: 100000
Завершение тренировки 

CPU times: user 1min 19s, sys: 8.2 s, total: 1min 27s
Wall time: 1min 25s


In [8]:
q_table[328]

array([ -2.41167308,  -2.27325184,  -2.40310349,  -2.36159906,
       -11.06859969, -10.20829148])

# Evaluate agent's performance after Q-learning

In [9]:
total_epochs, total_penalties = 0, 0
episodes = 100
frames = []

for ep in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0

    done = False

    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        # Поместите каждый визуализированный кадр в dict для анимации
        frames.append({
            'frame': env.render(mode='ansi'),
            'episode': ep,
            'state': state,
            'action': action,
            'reward': reward
            }
        )
        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Результат после {episodes} эпизодов:")
print(f"Среднее время на серию: {total_epochs / episodes}")
print(f"Среднее время на серию: {total_penalties / episodes}")

Результат после 100 эпизодов:
Среднее время на серию: 12.88
Среднее время на серию: 0.0


# Визуализация

In [10]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Episode: 99
Timestep: 1288
State: 85
Action: 5
Reward: 20
